In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import warnings
import sklearn
import sklearn.model_selection
np.set_printoptions(precision=3, suppress=True)  # Print as 0.001 instead of 9.876e-4
torch.set_printoptions(precision=3, sci_mode=False)




In [2]:
#adding match data from file

data = np.loadtxt('newmatchdata.csv',skiprows=1,delimiter=',')
X_data = np.array(data[:,1:]).astype(np.int32)
y_data = np.array(data[:,:1]).astype(np.int32)

#turn all data into feature vector
#feature vector creation
X_val_trn = torch.zeros((40000,138*2),dtype=torch.float32)
j = 0
for d in X_data:
    for i in range(len(d)):
        if(i < len(d)/2):
            h = d[i]
            X_val_trn[j][h] += 1
            
            
        else:
            h = d[i]
            X_val_trn[j][h+137] += 1
      
    j += 1



In [19]:

#sklearn train test split

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X_val_trn,y_data, test_size = 0.2, shuffle= True)

#split train into train/validation set

X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(X_train,y_train, test_size = 0.15, shuffle= True)

X_train = torch.tensor(X_train,dtype=torch.float32)
y_train = torch.tensor(y_train,dtype=torch.float32)

X_val = torch.tensor(X_val,dtype=torch.float32)

y_val = torch.tensor(y_val,dtype=torch.float32)

X_test = torch.tensor(X_test,dtype=torch.float32)
y_test = torch.tensor(y_test,dtype=torch.float32)
#sklearn train test split
#use test set to calculate error, CCE, accuracy etc. 

C:\Users\david\AppData\Local\Temp\ipykernel_16504\337929334.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train,dtype=torch.float32)
C:\Users\david\AppData\Local\Temp\ipykernel_16504\337929334.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val = torch.tensor(X_val,dtype=torch.float32)
C:\Users\david\AppData\Local\Temp\ipykernel_16504\337929334.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test,dtype=torch.float32)


In [4]:

#model creation
#addition of multiple hidden layers and drop rate to help with regularization
print(torch.cuda.is_available())
model = torch.nn.Sequential(
    torch.nn.Linear(276,100),
    torch.nn.Dropout(),
    torch.nn.ReLU(),
    torch.nn.Linear(100,50),
    torch.nn.Dropout(),
    torch.nn.ReLU(),
    torch.nn.Linear(50,1)
)

model2 = torch.nn.Sequential(
    torch.nn.Linear(276,1)
)



True


In [25]:
model2 = torch.nn.Sequential(
    torch.nn.Linear(276,128),
    #torch.nn.BatchNorm1d(128),
    torch.nn.Dropout(),
    torch.nn.ReLU(),
    
    torch.nn.Linear(128,64),
    torch.nn.Dropout(),
    torch.nn.ReLU(),
    # torch.nn.Dropout(p=0.2),
    torch.nn.Linear(64,32),
    torch.nn.Dropout(),
    torch.nn.ReLU(),
    torch.nn.Linear(32,16),
    torch.nn.Dropout(),
    torch.nn.ReLU(),
    torch.nn.Linear(16,1)
)

if torch.cuda.is_available(): 
 dev = "cuda:0" 
else: 
 dev = "cpu" 
device = torch.device(dev)

model2.to(device)


#loss function and optimizer
loss = torch.nn.BCEWithLogitsLoss()
optim = torch.optim.Adam(model2.parameters(),lr=1e-4, weight_decay=1e-3)

#epoch number
num_epoch = 500
next_epoch = 1
batch_size = 1000

valid_loss = 100000000000
loss_flag = False

#training loop
for epoch in range(next_epoch, next_epoch+num_epoch):
    model2.train()
    
    
    # Make an entire pass (an 'epoch') over the training data in batch_size chunks
    for i in range(0, len(X_train), batch_size):        
        X = X_train[i:i+batch_size].to(device)     # Slice out a mini-batch of features
        y = y_train[i:i+batch_size].to(device)     # Slice out a mini-batch of targets
        
        

        y_pred = model2(X)                   # Make predictions (final-layer activations)
        
        l = loss(y_pred, y)                 # Compute loss with respect to predictions
        
        model2.zero_grad()                   # Reset all gradient accumulators to zero (PyTorch thing)
        l.backward()                        # Compute gradient of loss wrt all parameters (backprop!)
        optim.step()                    # Use the gradients to take a step with SGD.
        
    print("Epoch %2d: loss on final training batch: %.4f" % (epoch, l.item()))
    # pred = torch.sign(model(X_val))
    # acc = torch.mean((pred == y_val).float())
    
    model2.eval()

    #create dire query
    
    #validation set calculations
    dire_X =  torch.index_select(X_val, 1, torch.LongTensor([*range(138,276)]))
    dire_X = torch.cat((dire_X,torch.index_select(X_val, 1, torch.LongTensor([*range(0,138)]))),1).to(device)

    dire_pred = (model2(dire_X) >= 0).float()
    
    
    rad_pred = (model2(X_val.to(device)) >= 0).float()
    

    overall_prob = (((rad_pred + (1 - dire_pred))/2) > 0.5).float()
    
    val_acc = torch.mean((overall_prob.to('cpu') == y_val.to('cpu')).float())

    #training set calculations
    dire_X_train =  torch.index_select(X_train, 1, torch.LongTensor([*range(138,276)]))
    dire_X_train = torch.cat((dire_X_train,torch.index_select(X_train, 1, torch.LongTensor([*range(0,138)]))),1).to(device)

    dire_pred = (model2(dire_X_train) >= 0).float()
    
    rad_pred = (model2(X_train.to(device)) >= 0).float()
    

    overall_prob = (((rad_pred + (1 - dire_pred))/2) > 0.5).float()
    
    train_acc = torch.mean((overall_prob.to('cpu') == y_train.to('cpu')).float())
    
    print("Epoch %2d: accuracy on validation set: %.4f" % (epoch, val_acc))
    print("Epoch %2d: accuracy on training set: %.4f" % (epoch, train_acc))
    
    print("Epoch %2d: loss on validation set: %.4f" % (epoch, loss(model2(X_val.to(device)), y_val.to(device))))
    new_val_loss = loss(model2(X_val.to(device)), y_val.to(device))

    if new_val_loss > valid_loss:
       pass

    valid_loss = new_val_loss
    
   

next_epoch = epoch+1

Epoch  1: loss on final training batch: 0.6955
Epoch  1: accuracy on validation set: 0.4977
Epoch  1: accuracy on training set: 0.4920
Epoch  1: loss on validation set: 0.6953
Epoch  2: loss on final training batch: 0.6917
Epoch  2: accuracy on validation set: 0.4977
Epoch  2: accuracy on training set: 0.4920
Epoch  2: loss on validation set: 0.6949
Epoch  3: loss on final training batch: 0.6905
Epoch  3: accuracy on validation set: 0.4977
Epoch  3: accuracy on training set: 0.4920
Epoch  3: loss on validation set: 0.6946
Epoch  4: loss on final training batch: 0.6907
Epoch  4: accuracy on validation set: 0.4977
Epoch  4: accuracy on training set: 0.4920
Epoch  4: loss on validation set: 0.6944
Epoch  5: loss on final training batch: 0.6913
Epoch  5: accuracy on validation set: 0.4977
Epoch  5: accuracy on training set: 0.4920
Epoch  5: loss on validation set: 0.6942
Epoch  6: loss on final training batch: 0.6896
Epoch  6: accuracy on validation set: 0.4977
Epoch  6: accuracy on traini

In [17]:
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X_val_trn,y_data, test_size = 0.2, shuffle= True)

mlp = MLPClassifier(hidden_layer_sizes = 4,activation='relu',solver='adam',batch_size=16,learning_rate_init=0.05,max_iter=3000,random_state=0)# Your code here
log = LogisticRegression(max_iter=300)
mlp.fit(X_train,y_train.ravel())
log.fit(X_train,y_train.ravel())

print('training accuracy: ',mlp.score(X_train,y_train)*100,'%')
print('held-out accuracy (testing): ',mlp.score(X_test,y_test)*100,'%')

dire_X =  torch.index_select(X_test, 1, torch.LongTensor([*range(138,276)]))
dire_X = torch.cat((dire_X, torch.index_select(X_test, 1, torch.LongTensor([*range(0,138)]))),1)



dire_prob = log.score(dire_X,y_test)
rad_prob = log.score(X_test,y_test)

overall_prob = ((rad_prob + (1 - dire_prob))/2)

print('log training accuracy: ',log.score(X_train,y_train)*100,'%')
print('log held-out accuracy (testing): ',log.score(X_test,y_test)*100,'%')

print('log overall accuracy: ',overall_prob*100,'%')

for k in range(2,10):
    scores = sklearn.model_selection.cross_val_score(log,X_test,y_test.ravel(),cv=k) 
    print("held-out accuracy (%d-fold):   %.1f%%" % (k, scores.mean()*100))


training accuracy:  50.64062499999999 %
held-out accuracy (testing):  50.9875 %
log training accuracy:  58.203125 %
log held-out accuracy (testing):  57.037499999999994 %
log overall accuracy:  53.82499999999999 %
held-out accuracy (2-fold):   54.1%
held-out accuracy (3-fold):   55.1%
held-out accuracy (4-fold):   55.3%
held-out accuracy (5-fold):   55.8%
held-out accuracy (6-fold):   55.8%
held-out accuracy (7-fold):   55.5%
held-out accuracy (8-fold):   55.6%
held-out accuracy (9-fold):   55.6%
held-out accuracy (10-fold):   55.6%
held-out accuracy (11-fold):   55.8%
held-out accuracy (12-fold):   55.7%
held-out accuracy (13-fold):   55.6%
held-out accuracy (14-fold):   55.8%
held-out accuracy (15-fold):   55.5%
held-out accuracy (16-fold):   55.6%
held-out accuracy (17-fold):   55.5%
held-out accuracy (18-fold):   55.5%
held-out accuracy (19-fold):   55.4%


In [24]:
#test accuracy on test set 
dire_X =  torch.index_select(X_test, 1, torch.LongTensor([*range(138,276)]))
dire_X = torch.cat((dire_X,torch.index_select(X_test, 1, torch.LongTensor([*range(0,138)]))),1).to(device)
dire_pred = (model2(dire_X) >= 0).float()
rad_pred = (model2(X_test.to(device)) >= 0).float()


overall_prob = (((rad_pred + (1 - dire_pred))/2) > 0.5).float()
acc = torch.mean((overall_prob.to(device) == y_test.to(device)).float())
print(acc)

tensor(0.520, device='cuda:0')
